In [2]:
#22.08.2024
#Updated code

import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Open the website
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
driver.maximize_window()
time.sleep(5)  # Wait for the page to load

# Function to close the signup popup if it appears
def close_signup_popup():
    try:
        close_button = driver.find_element(By.CLASS_NAME, "icon-cross")
        driver.execute_script("arguments[0].click();", close_button)
        time.sleep(2)  # Wait for the popup to close
        print("Signup popup closed.")
    except NoSuchElementException:
        # If the popup is not found, we just continue
        pass

def scroll_page():
    last_height=driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(5)
        new_height=driver.execute_script("return document.body.scrollHeight")
        if last_height==new_height:
            break
        last_height=new_height
    return last_height

# Locate the route element by its class name 'route'
def get_links_and_text():
    roots = driver.find_elements(By.CLASS_NAME, 'route')
    root_names = [i.text for i in roots]
    links = [element.get_attribute('href') for element in roots]
    return root_names, links

def go_to_next_page():
    wait = WebDriverWait(driver, 10)
    try:
        active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
        active_page_number = active_page_element.text
        next_page_number = str(int(active_page_number) + 1)
        
        next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']")))
        next_page_button.click()
        time.sleep(5)  # Wait for the next page to load
    except Exception as e:
        print(f"Error while clicking next page: {e}")

# Retrieve data from all pages
all_root_names = []
all_links = []

for i in range(5):  # Assuming there are multiple pages
    root_names, links = get_links_and_text()
    all_root_names.extend(root_names)
    all_links.extend(links)
    if i < 1:  # Adjust based on the number of pages
        go_to_next_page()

# Navigate to each route and extract bus data
bus_data = []

for route_name, link in zip(all_root_names, all_links):
    driver.get(link)
    time.sleep(5)  # Wait for the page to load

    scroll_page() 
    # Find all "View Buses" buttons
    view_buses_buttons = driver.find_elements(By.CLASS_NAME, "button")
   
    for button in view_buses_buttons:
        try:
            driver.execute_script("arguments[0].click();", button)
            time.sleep(5)  # Wait for buses to load
            
            # Close the signup popup if it appears
            close_signup_popup()

            
            scroll_page()

            # Extract bus details
            bus_elements = driver.find_elements(By.CLASS_NAME, 'row-sec')

            for element in bus_elements:
                try:
                    bus_name = element.find_element(By.CLASS_NAME, 'travels').text.strip()
                    bus_type = element.find_element(By.CLASS_NAME, 'bus-type').text.strip()
                    departing_time = element.find_element(By.CLASS_NAME, 'dp-time').text.strip()
                    duration = element.find_element(By.CLASS_NAME, 'dur').text.strip()
                    reaching_time = element.find_element(By.CLASS_NAME, 'bp-time').text.strip()
                    star_rating = element.find_element(By.XPATH, './/div[@class="rating-sec lh-24"]//span').text.strip()
                    try:
                        price_element = element.find_element(By.CSS_SELECTOR, 'span.f-19.f-bold')
                        price = price_element.text.strip()
                    except NoSuchElementException:
                        price = "Price not available"

                    seats_available = element.find_element(By.CLASS_NAME, 'seat-left').text.strip().split()[0]

                    bus_data.append({
                        "Route Name": route_name,
                        "Route Link": link,
                        "Bus Name": bus_name,
                        "Bus Type": bus_type,
                        "Departing Time": departing_time,
                        "Duration": duration,
                        "Reaching Time": reaching_time,
                        "Star Rating": star_rating,
                        "Price": price,
                        "Seats Available": int(seats_available)
                    })
                except Exception as e:
                    print(f"Error extracting bus data: {e}")

        except Exception as e:
            print(f"Error clicking 'View Buses' button: {e}")

# Save the extracted bus data to a CSV file
csv_file_path = "updated.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=bus_data[0].keys())
    writer.writeheader()
    writer.writerows(bus_data)

print(f"Data has been saved to {csv_file_path}")

# Close the browser
time.sleep(5)
driver.quit()


Error extracting bus data: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="rating-sec lh-24"]//span"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66221EEB2+31554]
	(No symbol) [0x00007FF662197EE9]
	(No symbol) [0x00007FF66205872A]
	(No symbol) [0x00007FF6620A8434]
	(No symbol) [0x00007FF6620A853C]
	(No symbol) [0x00007FF66209BBAC]
	(No symbol) [0x00007FF6620CD06F]
	(No symbol) [0x00007FF66209BA76]
	(No symbol) [0x00007FF6620CD240]
	(No symbol) [0x00007FF6620EC977]
	(No symbol) [0x00007FF6620CCDD3]
	(No symbol) [0x00007FF66209A33B]
	(No symbol) [0x00007FF66209AED1]
	GetHandleVerifier [0x00007FF662528B2D+3217341]
	GetHandleVerifier [0x00007FF662575AF3+3532675]
	GetHandleVerifier [0x00007FF66256B0F0+3489152]
	GetHandleVerifier [0x00007FF6622CE786+750614]
	(No s

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF66221EEB2+31554]
	(No symbol) [0x00007FF662197EE9]
	(No symbol) [0x00007FF66205872A]
	(No symbol) [0x00007FF66202D995]
	(No symbol) [0x00007FF6620D44D7]
	(No symbol) [0x00007FF6620EC051]
	(No symbol) [0x00007FF6620CCDD3]
	(No symbol) [0x00007FF66209A33B]
	(No symbol) [0x00007FF66209AED1]
	GetHandleVerifier [0x00007FF662528B2D+3217341]
	GetHandleVerifier [0x00007FF662575AF3+3532675]
	GetHandleVerifier [0x00007FF66256B0F0+3489152]
	GetHandleVerifier [0x00007FF6622CE786+750614]
	(No symbol) [0x00007FF6621A376F]
	(No symbol) [0x00007FF66219EB24]
	(No symbol) [0x00007FF66219ECB2]
	(No symbol) [0x00007FF66218E17F]
	BaseThreadInitThunk [0x00007FF831227374+20]
	RtlUserThreadStart [0x00007FF831E9CC91+33]
